## Access HERE API traffic data and saving to shapefile

#### Install herepy package

In [1]:
!pip install herepy -q #contextily rioxarray geocube -q

In [1]:
import herepy
import requests
import json
import os

import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import shapely as shp
import geopandas as gpd
import shutil
from src.data.utils import bounds
from src.utils import keys

Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.


#### HERE Traffic API request

In [3]:
area = bounds()

In [5]:
key = keys()['HERE']['key']

In [6]:
traffic_url = 'https://traffic.ls.hereapi.com/traffic/6.2/flow.json?\
apiKey=%s\
&bbox=%s,%s;%s,%s\
&responseattributes=sh,fc'%(key, area[3], area[0], area[1], area[2])

In [7]:
now = dt.now()
now = now.strftime('%Y%m%d_%H%M')
now

'20210811_2019'

In [8]:
%%time
api_out = requests.get(traffic_url)
traffic = json.loads(api_out.content)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
%%time 

out = []

for it in traffic.get('RWS'):
    for rw in (it.get('RW')):
        for fis in (rw.get('FIS')):
            for fi in fis.get('FI'):
                DE = (fi.get('TMC').get('DE'))
                shape = fi.get('SHP')
                PC = fi.get('TMC').get('PC')
                QD = fi.get('TMC').get('QD')
                LE = fi.get('TMC').get('LE')
                
                zs = []

                for z in shape:
                    zs.append([[*map(float, x.split(','))][::-1] for x in z.get('value')[0].split(' ')[:-1]])

                line = shp.geometry.MultiLineString(zs)
                if len(fi.get('CF')) == 1:
                    SU = fi.get('CF')[0].get('SU')
                    SP = fi.get('CF')[0].get('SP')
                    JF = fi.get('CF')[0].get('JF')
                    FF = fi.get('CF')[0].get('FF')
                    CN = fi.get('CF')[0].get('CN')
                    
                else:
                    print('ERROR.')
                    
                out.append((DE, PC, QD, LE, SU, SP, JF, FF, CN, line))

In [ ]:
DE, PC, QD, LE, SU, SP, JF, FF, CN, geom = zip(*out)

In [ ]:
gdf = gpd.GeoDataFrame(data=zip(DE, PC, QD, LE, SU, SP, JF, FF, CN), 
                       geometry=list(geom), 
                       columns=('DE', 'PC', 'QD', 'LE', 'SU', 'SP', 'JF', 'FF', 'CN'))
gdf.head()

In [ ]:
print('%s'%gdf.shape[0], ' rows of data')

In [ ]:
%%time 

fig, ax = plt.subplots(1,1)

gdf.plot(ax=ax, column='JF', legend=True, lw=0.25)
# countries.plot(ax=ax, fc='#bebebe', ec='black', zorder=-10)
ax.set_xlim(area[0]-1, area[2]+1)
ax.set_ylim(area[1]-1, area[3]+1)
plt.tight_layout()

In [ ]:
gdf.to_file('here_traffic_', driver='ESRI Shapefile')
shutil.make_archive('data/traffic/4_43_10_49/%s'%now, 'zip', './here_traffic_/')
!rm -rf 'here_traffic_'

#### Run HERE traffic API request periodically to store temporal data

In [9]:
import time

In [10]:
gap = 20 # in minutes

In [ ]:
for i in range(12):
    
    now = dt.now()
    now = now.strftime('%Y%m%d_%H%M')
    
    print('Iteration: %s'%i)
    print(dt.now().strftime('%H:%M:%S'), '\n')
    
    api_out = requests.get(traffic_url)
    traffic = json.loads(api_out.content)
    
    out = []

    for it in traffic.get('RWS'):
        for rw in (it.get('RW')):
            for fis in (rw.get('FIS')):
                for fi in fis.get('FI'):
                    DE = (fi.get('TMC').get('DE'))
                    shape = fi.get('SHP')
                    PC = fi.get('TMC').get('PC')
                    QD = fi.get('TMC').get('QD')
                    LE = fi.get('TMC').get('LE')

                    zs = []

                    for z in shape:
                        zs.append([[*map(float, x.split(','))][::-1] for x in z.get('value')[0].split(' ')[:-1]])

                    line = shp.geometry.MultiLineString(zs)
                    if len(fi.get('CF')) == 1:
                        SU = fi.get('CF')[0].get('SU')
                        SP = fi.get('CF')[0].get('SP')
                        JF = fi.get('CF')[0].get('JF')
                        FF = fi.get('CF')[0].get('FF')
                        CN = fi.get('CF')[0].get('CN')

                    else:
                        print('ERROR.')

                    out.append((DE, PC, QD, LE, SU, SP, JF, FF, CN, line))
    
    DE, PC, QD, LE, SU, SP, JF, FF, CN, geom = zip(*out)
    
    gdf = gpd.GeoDataFrame(data=zip(DE, PC, QD, LE, SU, SP, JF, FF, CN), 
                       geometry=list(geom), 
                       columns=('DE', 'PC', 'QD', 'LE', 'SU', 'SP', 'JF', 'FF', 'CN'))
    
    gdf.to_file('here_traffic_', driver='ESRI Shapefile')
    shutil.make_archive('data/traffic/4_43_10_49/%s'%now, 'zip', './here_traffic_/')
    !rm -rf 'here_traffic_'
    
    time.sleep(50*gap)

Iteration: 0
17:36:32 

Iteration: 1
17:55:40 

Iteration: 2
18:14:57 

Iteration: 3
18:34:16 

Iteration: 4
18:53:31 



___

#### Test

In [ ]:
df1 = gpd.read_file('zip://data/traffic/20210703_2038.zip')
df2 = gpd.read_file('zip://data/traffic/20210704_0005.zip')

In [ ]:
df2.head()